# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv('../output_data/cities_dataframe.csv')
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Avarua,-21.2078,-159.7750,29.00,74,20,3.09,CK,1613263913
1,Plettenberg bay,-34.0527,23.3716,21.96,93,88,2.30,ZA,1613264019
2,Siniscola,40.5719,9.6922,5.00,65,40,6.69,IT,1613264019
3,Provideniya,64.3833,-173.3000,-11.42,92,100,7.90,RU,1613263881
4,Codrington,-38.2667,141.9667,16.58,65,100,4.64,AU,1613264020
...,...,...,...,...,...,...,...,...,...
569,San pedro de lloc,-7.4328,-79.5058,23.90,66,98,6.40,PE,1613264123
570,Ormara,25.2088,64.6357,20.28,63,0,3.34,PK,1613264123
571,Vila,42.0304,-8.1588,9.44,95,99,0.89,PT,1613264123
572,Thinadhoo,0.5333,72.9333,27.96,73,16,3.72,MV,1613264123


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

locations = cities_df[['Lat','Lng']]
humidity = cities_df['Humidity'].astype('float')

In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=10,
                                point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#min(cities_df['cloudiness'])
wc_df = cities_df[(cities_df['Max Temp'] <= 0) & (cities_df['Wind Speed'] >= 7.5) & (cities_df['Cloudiness'] >= 60)]\
                .reset_index(drop=True)
wc_df.columns

Index(['City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date'],
      dtype='object')

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = wc_df[['City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date']]
hotel_df['Hotel Name'] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Provideniya,64.3833,-173.3000,-11.42,92,100,7.90,RU,1613263881,
1,Petropavlivka,48.4564,36.4367,-11.30,92,100,7.69,UA,1613264024,
2,Tokmak,47.2514,35.7058,-9.19,89,99,7.63,UA,1613264049,
3,Nuuk,64.1835,-51.7216,-10.00,73,90,8.23,GL,1613264052,
4,Beringovskiy,63.0500,179.3167,-15.90,93,100,8.08,RU,1613264067,
5,Vardo,70.3705,31.1107,-2.15,86,100,7.65,NO,1613264087,
6,Lubyany,56.0378,51.4000,-6.00,86,90,11.00,RU,1613264089,
7,Sandpoint,48.2766,-116.5533,-5.00,25,90,9.26,US,1613263940,


In [7]:
from pprint import pprint

target_type = 'hotel'
target_radius = 5000
target_coordinates = ''
rankby = 'distance'

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index,rows in hotel_df.iterrows():
    cityname = rows['City']
    target_coordinates = str(rows['Lat']) + ', ' + str(rows['Lng'])
    
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }
    
    response = requests.get(base_url, params=params).json()
    results = response['results'][0]
    print(f'Retrieving Results for Index {index}: {cityname}')
    
    try:
        print(f'First {cityname} hotel to appear is {results["name"]}')
        
        hotel_df.loc[index, 'Hotel Name'] = results['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    
    print('-------'*10)


Retrieving Results for Index 0: Provideniya
First Provideniya hotel to appear is Provideniya
----------------------------------------------------------------------
Retrieving Results for Index 1: Petropavlivka
First Petropavlivka hotel to appear is Petropavlivka
----------------------------------------------------------------------
Retrieving Results for Index 2: Tokmak
First Tokmak hotel to appear is Tokmak
----------------------------------------------------------------------
Retrieving Results for Index 3: Nuuk
First Nuuk hotel to appear is Nuuk
----------------------------------------------------------------------
Retrieving Results for Index 4: Beringovskiy
First Beringovskiy hotel to appear is Beringovskii
----------------------------------------------------------------------
Retrieving Results for Index 5: Vardo
First Vardo hotel to appear is Vardo
----------------------------------------------------------------------
Retrieving Results for Index 6: Lubyany
First Lubyany hotel t

In [8]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Provideniya,64.3833,-173.3000,-11.42,92,100,7.90,RU,1613263881,Provideniya
1,Petropavlivka,48.4564,36.4367,-11.30,92,100,7.69,UA,1613264024,Petropavlivka
2,Tokmak,47.2514,35.7058,-9.19,89,99,7.63,UA,1613264049,Tokmak
3,Nuuk,64.1835,-51.7216,-10.00,73,90,8.23,GL,1613264052,Nuuk
4,Beringovskiy,63.0500,179.3167,-15.90,93,100,8.08,RU,1613264067,Beringovskii
5,Vardo,70.3705,31.1107,-2.15,86,100,7.65,NO,1613264087,Vardo
6,Lubyany,56.0378,51.4000,-6.00,86,90,11.00,RU,1613264089,Mariyskiy Saramak
7,Sandpoint,48.2766,-116.5533,-5.00,25,90,9.26,US,1613263940,Sandpoint


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))